# Test retrieve instance

In [ ]:
from deepdiff import DeepDiff
from laminhub_rest.core.account import delete_account
from laminhub_rest.core.instance import delete_instance
from laminhub_rest.connector import get_access_token
from laminhub_rest.main import client
from laminhub_rest.routers.instance import (
    get_instance_accounts,
    get_instance_by_name,
)
from laminhub_rest.connector import connect_hub
from laminhub_rest.utils._test import (
    create_test_auth,
    create_test_account,
    create_test_storage,
    create_test_instance,
    add_test_collaborator,
)
from laminhub_rest._clean_ci import clean_ci

## Parametrize 

In [ ]:
hub = connect_hub()

# Create account 1
auth_1 = create_test_auth()
access_token_1 = auth_1["access_token"]
account_1 = create_test_account(handle=auth_1["handle"], access_token=access_token_1)
account_id_1 = account_1["id"]
account_handle_ = account_1["handle"]

# Create account 2
auth_2 = create_test_auth()
access_token_2 = auth_2["access_token"]
account_2 = create_test_account(handle=auth_2["handle"], access_token=access_token_2)
account_id_2 = account_2["id"]
handle_2 = account_2["handle"]

# Create storage
storage = create_test_storage(access_token=access_token_1)

# Create instance
instance_1 = create_test_instance(storage_id=storage["id"], access_token=access_token_1)
instance_account_id_1 = instance_1["id"]
instance_name_1 = instance_1["name"]

# Add account 1 as an admin member

instance_collaborator_1 = add_test_collaborator(
    instance_id=instance_account_id_1,
    account_id=account_id_1,
    role="admin",
    access_token=access_token_1,
)

# Add account 2 as a member with read role

instance_collaborator_2 = add_test_collaborator(
    instance_id=instance_account_id_1,
    account_id=account_id_2,
    role="read",
    access_token=access_token_1,
)

## Fetch instance by name

In [ ]:
instance_expected = {
    "instance": {
        **instance_1,
        "storage": {"root": storage["root"]},
        "account": {"handle": account_handle_, "id": account_id_1},
    },
    "role": "admin",
}

### Call `get_instance_by_name`

In [ ]:
instance = get_instance_by_name(
    account_handle_, instance_1["name"], f"Bearer {access_token_1}"
)
assert str(instance) == str(instance_expected)

### Get method on `/instance/{handle}/{name}` route

In [ ]:
response = client.get(
    f"/instance/{account_handle_}/{instance_name_1}",
    headers={"authentication": f"Bearer {access_token_1}"},
)
assert str(response.json()) == str(instance_expected)

## Fetch instance's accounts

In [ ]:
instance_accounts_expected = {
    "accounts": [
        {
            **instance_collaborator_1,
            "account": account_1,
        },
        {
            **instance_collaborator_2,
            "account": account_2,
        },
    ],
    "role": "admin",
}

### Call `get_instance_accounts`

In [ ]:
instance_accounts = get_instance_accounts(
    account_handle_, instance_name_1, f"Bearer {access_token_1}"
)
assert DeepDiff(instance_accounts, instance_accounts_expected, ignore_order=True) == {}

### Get method on `/instance/resources/accounts/{handle}/{name}` route

In [ ]:
response = client.get(
    f"/instance/resources/accounts/{account_handle_}/{instance_name_1}",
    headers={"authentication": f"Bearer {access_token_1}"},
)
assert str(response.json()) == str(instance_accounts_expected)

## Clean up test assets

In [ ]:
delete_account(account_1["handle"], access_token_1)
delete_account(account_2["handle"], access_token_2)
delete_instance(
    owner=account_1["handle"], name=instance_1["name"], _access_token=access_token_1
)

In [ ]:
clean_ci()